# Preprocess Output
This script will comb through the FUSAR runways_data_set.csv data and produce as output a table of form:

| AIRPORT | DATE | TIME (15 min intervals) | # arrivals |
| ------- | ---- | ----------------------- | ---------- |

_In the final output, we will need to convert from TIME (15 min intervals) to BASETIME_BUCKET where BASETIME = TIME and BUCKET is 15 minute intervals._

In [5]:
import pandas as pd
import os
import math
from datetime import date, datetime
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def get_weekday(day):
    match day:
        case 0:
            return 'monday'
        case 1:
            return 'tuesday'
        case 2:
            return 'wednesday'
        case 3:
            return 'thursday'
        case 4:
            return 'friday'
        case 5:
            return 'saturday'
        case 6:
            return 'sunday'
        case _:
            return ''


In [3]:
def get_cyclical_time(time):
    hour = int(time[0:2])
    minute = int(time[2:])
    abs_time = (hour*3600)+(minute*60)
    return math.sin((abs_time/86400)*math.pi)

In [4]:
def sin(x, a):
    return math.sin((x/a)*math.pi)

In [5]:
def get_bucket(time):
    #given an arbitrary time t (hhmm), returns closest 15 minute interval
    #example: 0112 -> 0130
    hour = int(time[:2])
    minute = int(time[2:])
    if minute == 0:
        return str((hour)%24).zfill(2)+"00"
    elif minute <= 15 and minute > 0:
        return str(hour%24).zfill(2)+"15"
    elif minute > 15 and minute <= 30:
        return str(hour%24).zfill(2)+"30"
    elif minute > 30 and minute <= 45:
        return str(hour%24).zfill(2)+"45"
    elif minute > 45:
        return str((hour+1)%24).zfill(2)+"00"

In [6]:
#given time t, increases to next bucket
#ex: 0115 -> 0130
def increment_time(time):
    time = get_bucket(time)
    hour = int(time[:2])
    minute = int(time[2:])
    match minute:
        case 00:
            return str(hour%24).zfill(2)+"15"
        case 15:
            return str(hour%24).zfill(2)+"30"
        case 30:
            return str(hour%24).zfill(2)+"45"
        case 45:
            return str((hour+1)%24).zfill(2)+"00"
        case _:
            return None

In [7]:
def convert_timestamp(file, row):
    airport = file.split("_")[0]
    #date = (YYMMDD)
    date = row.split(" ")[0].split("-")[0][2:]+row.split(" ")[0].split("-")[1]+row.split(" ")[0].split("-")[2]
    time = get_bucket(row.split(" ")[-1].split(":")[0]+row.split(" ")[-1].split(":")[1])
    return airport, date, time              

In [8]:
Y = 2000 # dummy leap year to allow input X-02-29 (leap day)
seasons = [('winter', (date(Y,  1,  1),  date(Y,  3, 20))),
           ('spring', (date(Y,  3, 21),  date(Y,  6, 20))),
           ('summer', (date(Y,  6, 21),  date(Y,  9, 22))),
           ('autumn', (date(Y,  9, 23),  date(Y, 12, 20))),
           ('winter', (date(Y, 12, 21),  date(Y, 12, 31)))]
def get_season(now):
    if isinstance(now, datetime):
        now = now.date()
    now = now.replace(year=Y)
    return next(season for season, (start, end) in seasons
                if start <= now <= end)

In [21]:
def interpolate_nans(df):
    #given a df, for each column, replace nans with averages 
    # Use linear interpolation to fill NaNs, then limit the interpolation to "both"
    # so that it only fills NaNs that are bounded by known values above and below.
    df_interpolated = df.infer_objects(copy=False)
    df_interpolated = df_interpolated.apply(lambda col: col.interpolate(method='linear'))
    df_interpolated = df_interpolated.ffill().bfill()
    return df_interpolated


In [4]:
#returns a dataframe with cols [[airport_date_time][# arrivals]]
#target data
def parse_runways_data(folder_paths):
    extension = 'runways_data_set.csv'
    target_data = {}
    # Search for files in the specified folder
    for folder_path in folder_paths:
        for root, dirs, files in os.walk("data/"+folder_path):
            for file in files:
                if file.endswith(extension):
                    df = pd.read_csv(os.path.join(root, file))
                    #drop duplicates
                    df = df.drop_duplicates(subset='gufi', keep='last').reset_index(drop=True)
                    for ind, row in df.iterrows():
                        if not pd.isna(df.loc[ind, 'arrival_runway_actual_time']):
                            airport, date, time = convert_timestamp(file, df["arrival_runway_actual_time"][ind])
                            key = airport+"_"+date+"_"+time
                            if key in target_data.keys():
                                target_data[key]+=1
                            else:
                                target_data[key] = 0
    return pd.DataFrame.from_dict(target_data, orient='index', columns=['# arrivals'])

In [7]:
def parse_TFM_track_data(folder_paths):
    #slightly more complicated than I thought. There's multiple estimates for each flight, so we need to get the last estimate for each flight. 
    extension = 'TFM_track_data_set.csv'
    target_data = {}

    # Search for files in the specified folder
    for folder_path in folder_paths:
        for root, dirs, files in os.walk("data/"+folder_path):
            for file in files:
                if file.endswith(extension):
                    df = pd.read_csv(os.path.join(root, file))
                    #keep only the most recent estimate
                    df = df.drop_duplicates(subset='gufi', keep='last').reset_index(drop=True)
                    for ind, row in df.iterrows():
                        if not pd.isna(df.loc[ind, 'arrival_runway_estimated_time']):
                            airport, date, time = convert_timestamp(file, df["arrival_runway_estimated_time"][ind])
                            key = airport+"_"+date+"_"+time
                            if key in target_data.keys():
                                target_data[key]+=1
                            else:
                                target_data[key] = 0
    return pd.DataFrame.from_dict(target_data, orient='index', columns=['estimated # arrivals'])

In [12]:
#returns dataframe with cols [airport_date_time][][] 
#weather data
def parse_LAMP_data(folder_paths):
    extension = 'LAMP_data_set.csv'
    weather_data = {}
    #weather data is forecasts from timestamp t to n steps in the future. This only uses forecast n=1 steps into the future. 

    # Search for files in the specified folder
    for folder_path in folder_paths:
        for root, dirs, files in os.walk("data/"+folder_path):
            for file in files:
                if file.endswith(extension):
                    df = pd.read_csv(os.path.join(root, file))
                    for ind in range(0, len(df), 25):
                        for i in range(4):
                            if not pd.isna(df.loc[ind+i, 'forecast_timestamp']):
                                airport, date, time = convert_timestamp(file, df["forecast_timestamp"][ind+i])
                                #get the forecast data for each timestamp
                                temperature = df["temperature"][ind+i]
                                wind_direction = df["wind_direction"][ind+i]
                                wind_speed = df["wind_speed"][ind+i]
                                wind_gust = df["wind_gust"][ind+i]
                                cloud_ceiling = df["cloud_ceiling"][ind+i]
                                visibility = df["visibility"][ind+i]
                                cloud = df["cloud"][ind+i]
                                lightning_prob = df["lightning_prob"][ind+i]
                                precip = df["precip"][ind+i]
                                #set forecast for buckets 15,30,45,00
                                times = [time, increment_time(time), increment_time(increment_time(time)), increment_time(increment_time(increment_time(time)))]
                                keys = [airport+"_"+date+"_"+time for time in times]
                                for key in keys:
                                    weather_data[key] = [temperature, wind_direction, wind_speed, wind_gust, cloud_ceiling, visibility, cloud, lightning_prob, precip]
    return pd.DataFrame.from_dict(weather_data, orient='index', columns=['temperature', 'wind_direction', 'wind_speed', 'wind_gust', 'cloud_ceiling', 'visibility', 'cloud', 'lightning_prob', 'precip'])                

In [ ]:
def parse_METAR_data(folder_paths):
    df = pd.read_csv(folder_paths".csv")
    for ind, row in df.iterrows():
        if not pd.isna(df.loc[ind+i, 'forecast_timestamp']):
            df[]
    return pd.DataFrame.from_dict(weather_data, orient='index', columns=['temperature', 'wind_direction', 'wind_speed', 'wind_gust', 'cloud_ceiling', 'visibility', 'cloud', 'lightning_prob', 'precip'])    

In [ ]:
#for each row in df, add time, day, week, season data
#for now I'm doing a timeseries, but we could change this to one-hot in the future 
def extract_time_data(df):
    airports = []
    times= []
    weekdays = []
    days = []
    months = []
    seasons = []
    for ind, row in df.iterrows():
        split_str = ind.split("_")
        airport = split_str[0]
        date = split_str[1]
        time = get_cyclical_time(split_str[2])
        day = sin(datetime.strptime(date, '%y%m%d').timetuple().tm_yday, 365)
        weekday = get_weekday(datetime.strptime(date, '%y%m%d').weekday())
        month = date[2:4]
        season = get_season(datetime.strptime(date, '%y%m%d'))
        #append to list
        airports.append(airport)
        times.append(time)
        weekdays.append(weekday)
        days.append(day)
        months.append(month)
        seasons.append(season)
    df['Airport'] = airports
    df['Time'] = times
    df['Weekday'] = weekdays
    df['Day'] = days
    df['Month'] = months
    df['Season'] = seasons
    return df     

In [10]:
def get_train_data(columns_to_drop):
    #get training dataset
    folder_paths = ['FUSER_train/KATL', 
                    'FUSER_train/KCLT', 
                    'FUSER_train/KDEN', 
                    'FUSER_train/KDFW', 
                    'FUSER_train/KJFK', 
                    'FUSER_train/KMEM', 
                    'FUSER_train/KMIA', 
                    'FUSER_train/KORD',
                    'FUSER_train/KPHX',
                    'FUSER_train/KSEA'
                    ]
    #align weather data with target data using AIRPORT_DATE_TIME and index
    data = pd.concat([parse_LAMP_data(folder_paths), parse_runways_data(folder_paths), parse_TFM_track_data(folder_paths)], axis=1)
    data = data.sort_index()
    #extract time data from index
    data = extract_time_data(data)
    #one-hot encode
    data = pd.get_dummies(data, dtype=float)
    #move '# arrivals' to end of dataframe
    cols_at_end = ['# arrivals']
    data = data[[c for c in data if c not in cols_at_end] 
            + [c for c in cols_at_end if c in data]]
    #move 'estimated # arrivals' to front
    cols = ['estimated # arrivals'] + [col for col in data.columns if col != 'estimated # arrivals']
    data = data[cols]
    #interpolate nans
    data = interpolate_nans(data)
    for col in columns_to_drop:
        if col in data:
            data = data.drop(col, axis=1) 
    return data

In [9]:
def get_test_data(columns_to_drop):
    #get test dataset
    folder_paths = [
                    'FUSER_test/KATL', 
                    'FUSER_test/KCLT', 
                    'FUSER_test/KDEN', 
                    'FUSER_test/KDFW', 
                    'FUSER_test/KJFK', 
                    'FUSER_test/KMEM', 
                    'FUSER_test/KMIA', 
                    'FUSER_test/KORD',
                    'FUSER_test/KPHX',
                    'FUSER_test/KSEA']
    #align weather data with target data using AIRPORT_DATE_TIME and index
    data = pd.concat([parse_runways_data(folder_paths),parse_LAMP_data(folder_paths), parse_TFM_track_data(folder_paths)], axis=1)
    #sort by time:
    data = data.sort_index()
    #extract time data from index
    data = extract_time_data(data)
    #one-hot encode
    data = pd.get_dummies(data, dtype=float)
    #move '# arrivals' to end of dataframe
    cols_at_end = ['# arrivals']
    data = data[[c for c in data if c not in cols_at_end] 
            + [c for c in cols_at_end if c in data]]
    #move 'estimated # arrivals' to front
    cols = ['estimated # arrivals'] + [col for col in data.columns if col != 'estimated # arrivals']
    data = data[cols]
    #interpolate nans
    data = interpolate_nans(data)
    data = interpolate_nans(data)
    for col in columns_to_drop:
        if col in data:
            data = data.drop(col, axis=1) 
    return data

# Post Processing

In [17]:
def convert_id(ID):
    AIRPORT = ID.split("_")[0]
    DATE = ID.split("_")[1]
    BASETIME = ID.split("_")[2]
    BUCKET = int(ID.split("_")[3])
    for i in range(int(BUCKET/15)):
        BASETIME = increment_time(BASETIME)
    return AIRPORT+"_"+DATE+"_"+BASETIME

In [ ]:
def post_process(submission_format_df, predict_df):
    #given submission_format_df, grabs the indexes and finds a predict_df index that matches
    ids = []
    values = []
    for ind, row in submission_format_df.iterrows():
        ID = submission_format_df["ID"][ind]
        #convert from "AIPORT_DATE_BASETIME_BUCKET" -> "AIPORT_DATE_TIME"
        converted_ID = convert_id(ID)
        value = 0
        if converted_ID in predict_df.index:
            value = round(predict_df.loc[converted_ID]["Prediction"])
        ids.append(ID)
        values.append(value)
    return pd.DataFrame({"ID": ids, "Value": values})
        